In [1]:
import os
cur_dir = os.getcwd()
par_dir = os.path.abspath(os.path.join(cur_dir,os.pardir))
os.chdir(par_dir)

In [ ]:
import os
import numpy as np
import pandas as pd
from vae_earlystopping import EarlyStopping
import sys
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from model1.m2 import Model2
from loss1.l2 import loss2
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from kl_divergence.cyclical import gamma_cyclical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
### 구하기
import json

### Space Time 넣기  random_state
results = {
    "R2_Space_Time": [],      # bce_binary * mse (최종)
    "R2_Except_Space_Time": [],     # bce_prob * mse (sigmoid 가중)
}
for i in np.random.randint(1,100,size = 1):
    x_data = np.load('./data/metal.npy')
    c_data = np.load('./data/pre_re_fin.npy')
    
    x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = i,test_size = 0.4)
    x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = i, test_size = 0.5)
    x_scaler = MinMaxScaler()
    c_scaler = MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
    c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

    x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
    c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = [x_train,x_train,c_train]
    val_data = [x_val,x_val,c_val]
    test_data = [x_test,x_test,c_test]
    train_data = TensorDataset(*train_data)
    val_data = TensorDataset(*val_data)
    test_data = TensorDataset(*test_data)
    datas = [train_data,val_data,test_data]
    train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
    x_sample,x2_sample, c_sample= next(iter(train_loader))

    x_dim = x_sample.shape[1]
    x2_dim= x2_sample.shape[1]
    c_dim = c_sample.shape[1]

    model = Model2(x_dim,x2_dim,c_dim, z_dim=8,z2_dim = 8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 800
    gamma = gamma_cyclical(800, gamma_max=0.1, cycle_len=200,ratio=0.5)
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x,x2, c in train_loader:
            x,x2,c = x.to(device),x2.to(device),c.to(device)
            optimizer.zero_grad()
            bce_logit,x_hat, mu, logvar,mu2,logvar2 = model(x,x2,c)
            loss_dict = loss2(bce_logit,x_hat,x,x2,mu,logvar,mu2,logvar2,a=1,b=1,c=gamma,d=gamma)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x,v2_x, v_c in val_loader:
                v_x,v2_x,v_c = v_x.to(device),v2_x.to(device),v_c.to(device)
                v_bce_logit,v_x_hat, v_mu, v_logvar,v2_mu,v2_logvar = model(v_x,v2_x,v_c)
                loss_dict = loss2(v_bce_logit,v_x_hat, v_x,v2_x, v_mu,v_logvar,v2_mu,v2_logvar,a=1,b=1,c=gamma,d=gamma)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_x_hat = []
    all_x_true = []
    mse_logit_list = []
    x_true_list = []
    with torch.no_grad():
        for x_t,x2_t, c_t in test_loader:
            x_t,x2_t,c_t = x_t.to(device),x2_t.to(device),c_t.to(device)
            bce_logit_t,x_hat, mu_t,logvar_t,mu2_t,logvar2_t = model(x_t,x2_t,c_t)
            x_true = (x_t>0).float()
            mse_logit_list.append(x_hat.cpu().numpy())
            x_true_list.append(x_t.cpu().numpy())
            all_x_hat.append(bce_logit_t.detach().cpu())
            all_x_true.append(x_true.detach().cpu())
    all_x_hat_1 = torch.cat([a.flatten() for a in all_x_hat])
    all_x_true = torch.cat([a.flatten() for a in all_x_true])
    mse_logits = np.vstack(mse_logit_list)
    x_true = np.vstack(x_true_list)
    ### BCE-> 0,1 표현하여 MSE에 곱하기
    all_x_hat_tensor = torch.cat(all_x_hat, dim=0) # (Total_Samples, x_dim) 형태로 결합
    bce_logits_np = all_x_hat_tensor.cpu().numpy()
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    bce_prob = sigmoid(bce_logits_np)
    bce_binary = (bce_prob >= 0.5).astype(np.float32)
    x_hat_fin = x_scaler.inverse_transform(mse_logits)
    x_true = x_scaler.inverse_transform(x_true)
    final_x_sig = x_hat_fin*bce_prob
    from sklearn.metrics import r2_score,mean_squared_error
    r2_bce_mse_sig = r2_score(x_true.flatten(),final_x_sig.flatten())
    ##r2저장
    results["R2_Space_Time"].append(float(r2_bce_mse_sig))

### Space time 제외
for i in np.random.randint(1,100,size = 20):
    x_data = np.load('./data/metal.npy')
    c_data = np.load('./data/pre_re_exc_sp.npy')
    
    x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = i,test_size = 0.4)
    x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = i, test_size = 0.5)
    x_scaler = MinMaxScaler()
    c_scaler = MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
    c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

    x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
    c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = [x_train,x_train,c_train]
    val_data = [x_val,x_val,c_val]
    test_data = [x_test,x_test,c_test]
    train_data = TensorDataset(*train_data)
    val_data = TensorDataset(*val_data)
    test_data = TensorDataset(*test_data)
    datas = [train_data,val_data,test_data]
    train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
    x_sample,x2_sample, c_sample= next(iter(train_loader))

    x_dim = x_sample.shape[1]
    x2_dim= x2_sample.shape[1]
    c_dim = c_sample.shape[1]

    model = Model2(x_dim,x2_dim,c_dim, z_dim=8,z2_dim = 8).to(device)
    early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
    epochs = 800
    gamma = gamma_cyclical(800, gamma_max=0.1, cycle_len=200,ratio=0.5)
    for epoch in range(1,epochs+1):
        model.train()
        t_loss= 0
        for x,x2, c in train_loader:
            x,x2,c = x.to(device),x2.to(device),c.to(device)
            optimizer.zero_grad()
            bce_logit,x_hat, mu, logvar,mu2,logvar2 = model(x,x2,c)
            loss_dict = loss2(bce_logit,x_hat,x,x2,mu,logvar,mu2,logvar2,a=1,b=1,c=gamma,d=gamma)
            loss_dict['loss'].backward()
            optimizer.step()
            t_loss +=loss_dict['loss'].item()
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v_x,v2_x, v_c in val_loader:
                v_x,v2_x,v_c = v_x.to(device),v2_x.to(device),v_c.to(device)
                v_bce_logit,v_x_hat, v_mu, v_logvar,v2_mu,v2_logvar = model(v_x,v2_x,v_c)
                loss_dict = loss2(v_bce_logit,v_x_hat, v_x,v2_x, v_mu,v_logvar,v2_mu,v2_logvar,a=1,b=1,c=gamma,d=gamma)
                v_loss += loss_dict['loss'].item()
        avg_train_loss = t_loss/len(train_loader)
        avg_val_loss = v_loss/len(val_loader)

        if epoch % 20 ==0  or epoch ==2:
            print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
        if early_stopping(avg_val_loss,model):
            break

    early_stopping.load_best_model(model)
    model.eval()
    all_x_hat = []
    all_x_true = []
    mse_logit_list = []
    x_true_list = []
    with torch.no_grad():
        for x_t,x2_t, c_t in test_loader:
            x_t,x2_t,c_t = x_t.to(device),x2_t.to(device),c_t.to(device)
            bce_logit_t,x_hat, mu_t,logvar_t,mu2_t,logvar2_t = model(x_t,x2_t,c_t)
            x_true = (x_t>0).float()
            mse_logit_list.append(x_hat.cpu().numpy())
            x_true_list.append(x_t.cpu().numpy())
            all_x_hat.append(bce_logit_t.detach().cpu())
            all_x_true.append(x_true.detach().cpu())
    all_x_hat_1 = torch.cat([a.flatten() for a in all_x_hat])
    all_x_true = torch.cat([a.flatten() for a in all_x_true])
    mse_logits = np.vstack(mse_logit_list)
    x_true = np.vstack(x_true_list)
    ### BCE-> 0,1 표현하여 MSE에 곱하기
    all_x_hat_tensor = torch.cat(all_x_hat, dim=0) # (Total_Samples, x_dim) 형태로 결합
    bce_logits_np = all_x_hat_tensor.cpu().numpy()
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    bce_prob = sigmoid(bce_logits_np)
    bce_binary = (bce_prob >= 0.5).astype(np.float32)
    x_hat_fin = x_scaler.inverse_transform(mse_logits)
    x_true = x_scaler.inverse_transform(x_true)
    final_x_sig = x_hat_fin*bce_prob
    from sklearn.metrics import r2_score,mean_squared_error
    r2_bce_mse_sig = r2_score(x_true.flatten(),final_x_sig.flatten())
    ##각각의 r2저장
    results["R2_Except_Space_Time"].append(float(r2_bce_mse_sig))



Epoch [2/800]|Train:3.8442 |Val:3.3713
Epoch [20/800]|Train:1.8835 |Val:1.7281
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.1312 |Val:1.0727
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
Epoch [60/800]|Train:0.8750 |Val:0.8791
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [80/800]|Train:0.7418 |Val:0.7501
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of

C:\Users\User\AppData\Local\Temp\ipykernel_8632\2518069528.py:117: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch [2/800]|Train:3.7527 |Val:3.6325
Epoch [20/800]|Train:1.9449 |Val:1.9800
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.1431 |Val:1.1369
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [60/800]|Train:0.9350 |Val:0.9193
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [80/800]|Train:0.7571 |Val:0.7

C:\Users\User\AppData\Local\Temp\ipykernel_8632\2518069528.py:215: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch [2/800]|Train:3.7508 |Val:3.5711
Epoch [20/800]|Train:2.1032 |Val:2.1074
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.5188 |Val:1.5392
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [60/800]|Train:1.1065 |Val:1.1463
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
Epoch [80/800]|Train:0.8797 |Val:0.9618
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of

...

Epoch [2/800]|Train:3.7883 |Val:3.6277
Epoch [20/800]|Train:2.0848 |Val:2.1091
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.3780 |Val:1.3984
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [60/800]|Train:0.9559 |Val:1.0333
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [80/800]|Train:0.7800 |Val:0.8298
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of

C:\Users\User\AppData\Local\Temp\ipykernel_8632\2518069528.py:215: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch [2/800]|Train:3.7153 |Val:3.4797
Epoch [20/800]|Train:1.9693 |Val:1.9622
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.1671 |Val:1.2823
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [60/800]|Train:0.8643 |Val:0.9699
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch [80/800]|Train:0.7362 |Val:0.8387
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of

C:\Users\User\AppData\Local\Temp\ipykernel_8632\2518069528.py:215: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch [2/800]|Train:3.8626 |Val:3.5334
Epoch [20/800]|Train:2.0750 |Val:1.9789
EarlyStopping counter: 1 out of 40
Epoch [40/800]|Train:1.1442 |Val:1.0884
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
Epoch [60/800]|Train:0.8565 |Val:0.8713
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
E

In [ ]:
results

In [ ]:

save_path = "./results_Space_Presence.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Saved:", save_path)